In [ ]:
import numpy as np
import scipy.ndimage

In [ ]:
import numpy as np

def pad_with_last_val(array, pad_width):
    pad_before = pad_width[0]
    pad_after = pad_width[1]

    last_before = array[:, :1, :, :] # Take the first element along axis=1
    last_after = array[:, -1:, :, :] # Take the last element along axis=1

    array = np.concatenate([last_before]*pad_before + [array] + [last_after]*pad_after, axis=1)

    last_before = array[:, :, :1, :] # Take the first element along axis=2
    last_after = array[:, :, -1:, :] # Take the last element along axis=2

    array = np.concatenate([last_before]*pad_before + [array] + [last_after]*pad_after, axis=2)
    
    return array

# create a 4D numpy array
array = np.random.rand(3, 10, 10, 10)

# pad the array by repeating the last value 2 times at the start and 3 times at the end of the 2nd and 3rd axes
padded_array = pad_with_last_val(array, (2, 3))

print(padded_array.shape)  # Should print: (3, 15, 15, 10)


In [ ]:
# Specify the angle
angle = 30

# Specify the axes
axes = (1, 2)  # Rotate in the xy plane

# pad the array
padding = 15
padded_array = pad_with_last_val(flo, (padding, padding))

# Rotate the array
rotated_array = scipy.ndimage.rotate(padded_array, angle, axes=axes, reshape=False)[:, padding:-padding, padding:-padding, :]

plt.imshow(rotated_array[1, :, :, 30])
plt.colorbar()

In [ ]:
rotated_vector = np.zeros((rotated_array.shape[0], rotated_array.shape[1], rotated_array.shape[2], rotated_array.shape[3]))
rotated_vector[[0]] = -np.sin(angle*np.pi/180)*rotated_array[[1]] + np.cos(angle*np.pi/180)*rotated_array[[0]]
rotated_vector[[1]] = np.cos(angle*np.pi/180)*rotated_array[[1]] + np.sin(angle*np.pi/180)*rotated_array[[0]]
rotated_vector[[2]] = rotated_array[[2]]

In [ ]:
# shift elements along the 3rd dimension by 2 positions
shifting = 20
shifted_array = np.roll(flo, shift=shifting, axis=1)
plt.imshow(shifted_array[1, :, :, 30])
plt.colorbar()

In [ ]:
# inversion of directions
# Specify the angle
angle = 180  # Rotate by 45 degrees
# Specify the axes
axes = (1, 2)  # Rotate in the xy plane
# Rotate the array
inverted_array = scipy.ndimage.rotate(flo, angle, axes=axes, reshape=False)
inverted_array = inverted_array*np.reshape(np.array([-1, -1, 1]), (-1, 1, 1, 1))
plt.imshow(inverted_array[1, :, :, 30])
plt.colorbar()

In [ ]:
def shift_flow(flow, shifting, axis=1):
    return np.roll(flow, shift=shifting, axis=axis)

In [ ]:
def rotate_flow(flow, angle, axes=(1, 2), padding=15):
    padded_array = pad_with_last_val(flow, (padding, padding))
    rotated_array = scipy.ndimage.rotate(padded_array, angle, axes=axes, reshape=False)[:, padding:-padding, padding:-padding, :]
    
    # rotate velocity fields
    rotated_vector = np.zeros((rotated_array.shape[0], rotated_array.shape[1], rotated_array.shape[2], rotated_array.shape[3]))
    rotated_vector[[0]] = -np.sin(angle*np.pi/180)*rotated_array[[1]] + np.cos(angle*np.pi/180)*rotated_array[[0]]
    rotated_vector[[1]] = np.cos(angle*np.pi/180)*rotated_array[[1]] + np.sin(angle*np.pi/180)*rotated_array[[0]]
    rotated_vector[[2]] = rotated_array[[2]]
    
    return rotated_vector

In [ ]:
def invert_flow(flow):
    inverted_array = scipy.ndimage.rotate(flow, angle=180, axes=(1, 2), reshape=False)
    inverted_array = inverted_array*np.reshape(np.array([-1, -1, 1]), (-1, 1, 1, 1))
    return inverted_array

In [ ]:
inv_flow = invert_flow(flo)
shi_flow = shift_flow(inv_flow, shifting=20)
new_flow = rotate_flow(shi_flow, angle=-80)

In [ ]:
def transform_flow(flow):
    if np.random.rand()>0.5:
        flow = invert_flow(flow)
    
    shifting = int((np.random.rand()-0.5)*40)
    flow = shift_flow(flow, shifting)
    
    angle = (np.random.rand()-0.5)*180
    flow = rotate_flow(flow, angle)
    
    return flow

In [ ]:
num_processors = cpu_count()
with Pool(processes=num_processors) as p:
    result_list = p.map(process_flow, noiseless_flows)

# Remove None results if any
result_list = [result for result in result_list if result is not None]
new_flows = np.concatenate(result_list, axis=0) if result_list else None

In [ ]:
from multiprocessing import Pool, cpu_count
from multiprocessing import current_process
from tqdm.contrib.concurrent import process_map

In [ ]:
def process_flow(data):
    # Set a different numpy random seed for each worker process
    np.random.seed(os.getpid() + current_process()._identity[0])
    # data contains both the index and the actual flow
    index, flow = data
    try:
        return np.expand_dims(transform_flow(flow), axis=0)
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [ ]:
num_processors = cpu_count()
# Create a list of tuples containing both indices and the actual flow data
indices_and_flows = [(i, flow) for i, flow in enumerate(noiseless_flows)] * 2
result_list = process_map(process_flow, indices_and_flows, max_workers=num_processors)
# Remove None results if any
result_list = [result for result in result_list if result is not None]
new_flows = np.concatenate(result_list, axis=0) if result_list else None

In [ ]:
flows, segmentation_maps, velocity_scales, sdfs, velocity_scale_for_norm = generate_3dstenosis(
    nvox=64, 
    transforms=True, 
    data_aug=1
) 